#### Gist:
Self Organizing Map for MNIST dataset. SOM network grid can be changed by changing the values in the SOM_Network_Shape variables. Raw data shape is fixed for this problem. Custom colour are used to represent the labels 0 to 9, colour vector is in RGB format (8 bit representation) but normalised to unit length (See vector Saturation_Norm). 

In [ ]:
# Import Required Libraries
import numpy 
from matplotlib import pyplot as plt
from copy import deepcopy
from matplotlib import patches as patches
from LoadData import load_images, load_labels
from matplotlib.lines import Line2D

In [ ]:
# Generate Data
Raw_Data_Shape = numpy.array([60000, 784])
SOM_Network_Shape = numpy.array([50, 50])
Images = load_images('train-images-idx3-ubyte.gz')
Labels_Train = load_labels('train-labels-idx1-ubyte.gz')
Images_Norm = Images/numpy.linalg.norm(Images, axis=1).reshape(Raw_Data_Shape[0], 1)
W_Initial_Guess = numpy.random.uniform(0, 1, (SOM_Network_Shape[0]*SOM_Network_Shape[1], Raw_Data_Shape[1]))
W_Initial_Guess_Norm = W_Initial_Guess/numpy.linalg.norm(W_Initial_Guess, axis=1).reshape(SOM_Network_Shape[0]*SOM_Network_Shape[1], 1)
Index = numpy.mgrid[0:SOM_Network_Shape[0],0:SOM_Network_Shape[1]].reshape(2, SOM_Network_Shape[0]*SOM_Network_Shape[1]).T

In [ ]:
# Parameters 
Epoch = 0 
Max_Epoch = 20000
eta_0 = 0.1
eta_time_const = 1000
sigma_0 = numpy.max(SOM_Network_Shape) * 0.5
sigma_time_const = 1000/numpy.log10(sigma_0)
Saturation = numpy.array([[221, 65, 50], [107, 91, 149], [254, 132, 14], [103, 46, 159], [198, 33, 104], [255, 0, 0], 
                 [0, 110, 109], [0, 83, 156], [117, 81, 57], [191, 214, 65]])
Saturation_Norm = Saturation/numpy.linalg.norm(Saturation, axis=1).reshape(10, 1)

In [ ]:
# Required Functions
def winning_neuron(x, W):
    # Also called as Best Matching Neuron/Best Matching Unit (BMU)
    return numpy.argmin(numpy.linalg.norm(x - W, axis=1))

def update_weights(lr, var, x, W, Grid):
    i = winning_neuron(x, W)
    d = numpy.square(numpy.linalg.norm(Grid - Grid[i], axis=1))
    # Topological Neighbourhood Function
    h = numpy.exp(-d/(2 * var * var))
    W = W + lr * h[:, numpy.newaxis] * (x - W)
    return W

def decay_learning_rate(eta_initial, epoch, time_const):
    return eta_initial * numpy.exp(-epoch/time_const)

def decay_variance(sigma_initial, epoch, time_const):
    return sigma_initial * numpy.exp(-epoch/time_const)

In [ ]:
# Main Loop
W_new = deepcopy(W_Initial_Guess_Norm)
eta = deepcopy(eta_0)
sigma = deepcopy(sigma_0)
while Epoch <= Max_Epoch:
    # Update Weights
    i = numpy.random.randint(0, Raw_Data_Shape[0])
    W_new = update_weights(eta, sigma, Images_Norm[i], W_new, Index)
    # Print
#     print('Epoch: ', Epoch, ' Learning Rate: ', eta, ' Varinance: ', sigma, '\n')
    # Next...
    eta = decay_learning_rate(eta_0, Epoch, eta_time_const)
    sigma = decay_variance(sigma_0, Epoch, sigma_time_const)
    Epoch += 1
print('Optimal Weights Reached!!!')

In [ ]:
# Test
W_final = deepcopy(W_new)
Colour = numpy.zeros((SOM_Network_Shape[0]*SOM_Network_Shape[1], 3))
for i in range(0, Raw_Data_Shape[0]):
    bmu = winning_neuron(Images_Norm[i], W_final)
    Colour[bmu] = Saturation_Norm[Labels_Train[i]]

Zero_Pos = numpy.where(~Colour.any(axis=1))[0] # numpy.where(Colour[:, 0] == 0)[0]
for i in range(0, Zero_Pos.size):
    temp = numpy.array([])
    for j in range(0, Raw_Data_Shape[0]):
        a = numpy.linalg.norm(Images_Norm[j] - W_final[Zero_Pos[i]])
        temp = numpy.concatenate((temp, [a]), axis=0)
    bmu = numpy.argmin(temp)
    Colour[Zero_Pos[i]] = Saturation_Norm[Labels_Train[bmu]]

In [ ]:
# Plot
fig = plt.figure()
# setup axes
ax = fig.add_subplot(111, aspect='equal')
ax.set_xlim((0, SOM_Network_Shape[0]))
ax.set_ylim((0, SOM_Network_Shape[1]))
ax.set_title('Self-Organising Map after %d iterations' % Max_Epoch)

# plot the rectangles
i = 0
for x in range(0, SOM_Network_Shape[0]):
    for y in range(0, SOM_Network_Shape[1]):
        ax.add_patch(patches.Rectangle((x, y), 1, 1, facecolor=Colour[i], edgecolor='none'))
        i += 1
# Add legends
legend_elements = [Line2D([0], [0], color=Saturation_Norm[0], lw=4, label='0'), 
                  Line2D([0], [0], color=Saturation_Norm[1], lw=4, label='1'),
                  Line2D([0], [0], color=Saturation_Norm[2], lw=4, label='2'),
                  Line2D([0], [0], color=Saturation_Norm[3], lw=4, label='3'),
                  Line2D([0], [0], color=Saturation_Norm[4], lw=4, label='4'),
                  Line2D([0], [0], color=Saturation_Norm[5], lw=4, label='5'),
                  Line2D([0], [0], color=Saturation_Norm[6], lw=4, label='6'),
                  Line2D([0], [0], color=Saturation_Norm[7], lw=4, label='7'),
                  Line2D([0], [0], color=Saturation_Norm[8], lw=4, label='8'),
                  Line2D([0], [0], color=Saturation_Norm[9], lw=4, label='9')]
ax.legend(handles=legend_elements, loc='center left', bbox_to_anchor=(1, 0.5))
plt.savefig('Self-Organising Map.pdf')
plt.show()